In [7]:
from cornac.data import Reader
from cornac.eval_methods import RatioSplit
from cornac.models import MF
from collections import defaultdict

import numpy as np


# Cargar dataset
reader = Reader()
data = reader.read(fpath='datasets/ml-100k/u.data', fmt='UIRT', sep='\t')

# Separación entrenamiento/test
eval_method = RatioSplit(
    data=data,
    test_size=0.2,
    rating_threshold=0.0,
    exclude_unknowns=True,
    verbose=True,
    seed=42
)

# Entrenar modelo
model = MF(k=10, max_iter=50, learning_rate=0.01, verbose=True)
model.fit(eval_method.train_set)

# Historial de ítems vistos por usuario
train_set = eval_method.train_set
user_rated = {}

# Construir historial real desde el entrenamiento
user_indices, item_indices, _ = train_set.uir_tuple

for uid, iid in zip(user_indices, item_indices):
    user_rated.setdefault(uid, set()).add(iid)

# Contar interacciones por usuario en entrenamiento
interacciones_por_usuario = defaultdict(int)
for uid in train_set.uid_map.values():
    interacciones_por_usuario[uid] = len(user_rated.get(uid, []))

# Filtrar usuarios fríos (≤ 5 interacciones)
usuarios_frios = [uid for uid, n in interacciones_por_usuario.items() if n <= 20]
print(f"Usuarios fríos encontrados: {len(usuarios_frios)}")


rating_threshold = 0.0
exclude_unknowns = True
---
Training data:
Number of users = 943
Number of items = 1651
Number of ratings = 80000
Max rating = 5.0
Min rating = 1.0
Global mean = 3.5
---
Test data:
Number of users = 943
Number of items = 1651
Number of ratings = 19964
Number of unknown users = 0
Number of unknown items = 0
---
Total users = 943
Total items = 1651


  0%|          | 0/50 [00:00<?, ?it/s]

Optimization finished!
Usuarios fríos encontrados: 148


In [8]:
# Todos los ítems posibles (índices internos)
all_items_internal = set(train_set.iid_map.values())

TOP_N = 5

def get_top_n(model, train_set, user_rated, all_items_internal, top_n=5):
    top_n_dict = {}
    for uid in train_set.uid_map.values():
        seen = user_rated.get(uid, set())
        candidates = all_items_internal - seen
        scores = [(iid, model.score(uid, iid)) for iid in candidates]
        scores.sort(key=lambda x: x[1], reverse=True)
        top_n_dict[uid] = [iid for iid, _ in scores[:top_n]]
    return top_n_dict

top_n_mf = get_top_n(model, train_set, user_rated, all_items_internal)

top_n_mf_frios = {uid: recs for uid, recs in top_n_mf.items() if uid in usuarios_frios}


In [9]:
# Ground truth del test
test_set = eval_method.test_set
ground_truth = {}
for uid, iid, _ in zip(*test_set.uir_tuple):
    ground_truth.setdefault(uid, set()).add(iid)

# Precision@5
def precision_at_k(top_n, ground_truth, k=5):
    precisiones = []
    for uid, recs in top_n.items():
        gt = ground_truth.get(uid, set())
        if not gt:
            continue
        hits = sum([1 for iid in recs if iid in gt])
        precisiones.append(hits / k)
    return np.mean(precisiones)

# Cobertura
def cobertura(top_n, total_items):
    items_recomendados = set(iid for recs in top_n.values() for iid in recs)
    return len(items_recomendados) / total_items

# Métricas
precision_frios = precision_at_k(top_n_mf_frios, ground_truth)
cobertura_frios = cobertura(top_n_mf_frios, len(train_set.iid_map))

print(f"Precision@5 (usuarios fríos): {precision_frios:.4f}")
print(f"Cobertura (usuarios fríos): {cobertura_frios:.4f}")


Precision@5 (usuarios fríos): 0.0135
Cobertura (usuarios fríos): 0.0836


**Resultados para usuarios fríos**

| Métrica      | Valor  |
| ------------ | ------ |
| Precision\@5 | 0.0135 |
| Cobertura    | 0.0836 |

**Desempeño bajo en precisión**

* Un 0.0135 en Precision\@5 significa que, en promedio, menos del 2% de las recomendaciones para usuarios fríos aciertan con ítems relevantes.
* Esto confirma lo esperado: MF necesita historial previo para generar buenas recomendaciones.

**Cobertura relativamente buena**

* El modelo sigue recomendando ítems diversos, incluso para usuarios con poca información.
* Esto sugiere que MF aún explora, pero no lo hace con acierto para estos casos.

**Limitación del modelo colaborativo puro**

* MF se basa completamente en interacciones pasadas.
* Para usuarios nuevos, no tiene datos para construir sus vectores latentes de forma informativa.
* Resultado: las recomendaciones se acercan a lo aleatorio.